In [1]:
import numpy as np
import utils
import pandas as pd

In [2]:
synth_regular = utils.load_synthetic()
compas = utils.load_compas_alt()
RESPONSE = "score_factor"
N = 20

In [3]:
compas.keys()

dict_keys(['train', 'test'])

In [4]:
#compas["standard"]["train"].keys()

In [5]:
#Checking that the classes don't become too much more imbalanced when data is removed.

#Synthetic data
for key, value in synth_regular.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))

Key:  test
Caucasian = 1:  0.35185185185185186 
Caucasian = 0:  0.6481481481481481
Male = 1:  0.8183183183183184 
Male = 0:  0.1816816816816817
After 30% missing: 
Caucasian = 1:  0.29309090909090907 
Caucasian = 0:  0.7069090909090909
Male = 1:  0.8269090909090909 
Male = 0:  0.1730909090909091
Key:  train
Caucasian = 1:  0.3473263368315842 
Caucasian = 0:  0.6526736631684158
Male = 1:  0.811344327836082 
Male = 0:  0.18865567216391804
After 30% missing: 
Caucasian = 1:  0.2989172467130704 
Caucasian = 0:  0.7010827532869296
Male = 1:  0.8368136117556071 
Male = 0:  0.16318638824439288


In [6]:
#Compas data
"""for ver in ["standard", "violent"]:
    print("Compas dataset type: ", ver)"""
for key, value in compas.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("Score text: ", len(value[value["score_factor"]==1])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))
    print("Score text: ", len(value[value["score_factor"]==1])/len(value))


Key:  train
Caucasian = 1:  0.3388149939540508 
Caucasian = 0:  0.6611850060459492
Male = 1:  0.8099153567110037 
Male = 0:  0.19008464328899638
Score text:  0.44401451027811367
After 30% missing: 
Caucasian = 1:  0.32952924393723254 
Caucasian = 0:  0.6704707560627675
Male = 1:  0.8088445078459344 
Male = 0:  0.19115549215406563
Score text:  0.44401451027811367
Key:  test
Caucasian = 1:  0.3446244477172312 
Caucasian = 0:  0.6553755522827688
Male = 1:  0.8090328915071183 
Male = 0:  0.19096710849288168
Score text:  0.4491899852724595
After 30% missing: 
Caucasian = 1:  0.3361462728551336 
Caucasian = 0:  0.6638537271448663
Male = 1:  0.7883263009845288 
Male = 0:  0.21167369901547117
Score text:  0.4491899852724595


In [7]:
percentiles = [p for p in range(0,20,2)]+[p for p in range(20,60, 10)]
missing=["two_year_recid", "is_Caucasian", "gender_factor", "crime_factor"]
all_results = {}
for i in range(N):
    temp_results = {}
    for miss in missing:
        for sensitive in ["is_Caucasian", "gender_factor"]:
            try:
                synth_results = utils.test_bench(train = synth_regular["train"],test = synth_regular["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                                percentiles = percentiles)
                recid_results = utils.test_bench(train = compas["train"],test = compas["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                                percentiles = percentiles)
                #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                                #percentiles = percentiles)
                temp_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
                temp_results[miss+"_"+sensitive+"_"+"recid"] = recid_results 
                #all_results[miss+"_"+sensitive+"_"+"v_recid"] = v_recid_results
            except Exception as e:
                print("Exception: ", e)
                print("Parameters: ", sensitive+"_"+miss)
    all_results[i] = temp_results 
        

 25%|██▌       | 1/4 [00:52<02:36, 52.26s/it]


KeyboardInterrupt: 

In [ ]:
import json
from pathlib import Path
with open(Path("raw_data/multi.json"), 'w') as f:
            json.dump(all_results, f)

In [ ]:
for key, res in all_results.items():
    utils.plotting_cf(["log_reg", "rf_cat", "svm", "knn"],["cca", "mice_def", "mean"], res, key+"/")
    utils.plotting_others(res, key+"/")

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


<Figure size 1332x756 with 0 Axes>